In [1]:
# define dependencies
import pandas as pd
from sqlalchemy import create_engine
from sql_key import post_gres_pw

## Loading the Data Sets

In [3]:
# store COVID dataset into DataFrame
covid_csv = "datasets/1_county_level_confirmed_cases.csv"
covid_df = pd.read_csv(covid_csv, dtype=str)
covid_df.head()

,last_update,location_type,state,county_name,county_name_long,fips_code,lat,lon,NCHS_urbanization,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000
0,2020-09-09 01:28:59 UTC,county,Alabama,Autauga,"Autauga, Alabama, US",01001,32.53952745,-86.64408227,Medium metro,55200.0,1385,2509.06,23,41.67
1,2020-09-09 01:28:59 UTC,county,Alabama,Baldwin,"Baldwin, Alabama, US",01003,30.72774991,-87.72207058,Small metro,208107.0,4609,2214.73,42,20.18
2,2020-09-09 01:28:59 UTC,county,Alabama,Barbour,"Barbour, Alabama, US",01005,31.868263,-85.3871286,Non-core,25782.0,617,2393.14,7,27.15
3,2020-09-09 01:28:59 UTC,county,Alabama,Bibb,"Bibb, Alabama, US",01007,32.99642064,-87.1251146,Large fringe metro,22527.0,562,2494.78,6,26.63
4,2020-09-09 01:28:59 UTC,county,Alabama,Blount,"Blount, Alabama, US",01009,33.98210918,-86.56790593,Large fringe metro,57645.0,1071,1857.92,12,20.82


In [4]:
# store poverty dataset into DataFrame
poverty_csv = "datasets/poverty_csv.csv"
poverty_df = pd.read_csv(poverty_csv)
poverty_df.head()

,Table with column headers in rows 3 and 4,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87
0,Table: Estimated Population in Poverty Univer...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,July 2018 ACS-Like Poverty Universe for 2018 E...,NaN,NaN,NaN,July 2017 ACS-Like Poverty Universe for 2017 E...,NaN,...,NaN,NaN,July 2000 CPS-Like Poverty Universe for IY 199...,NaN,NaN,NaN,July 1999 CPS-Like Poverty Universe for IY 199...,NaN,NaN,NaN
2,State FIPS code,County FIPS code,Name,State Postal Code,"Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related",...,"Poverty Universe, Age 0-17","Poverty Universe, Age 0-4","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4"
3,00,000,United States,US,"319,184,033","52,529,919","72,163,269","19,301,529","317,741,588","52,669,201",...,"71,741,141","19,181,906",276207757,51642359,71684956,18968750,271059449,51060953,71338364,19382484
4,01,000,Alabama,AL,"4,763,811","781,913","1,069,994","284,188","4,752,519","790,771",...,"1,104,080","296,196",4368014,804291,1120718,293558,4348444,789510,1088427,295264


## Cleaning/Formatting the DataFrames

### Cleaning the COVID DF
* Columns removed: 'last_update', 'location_type', 'county_name_long',	'NCHS_urbanization'.
* last_update: since the file will be used one time, stays on record the last update date but the column is removed.
* The data set was loaded in string format. Change the columns 'total_population	confirmed	confirmed_per_100000	deaths	deaths_per_100000' 

In [5]:
covid_data = covid_df[['state','county_name','fips_code','lat','lon','total_population','confirmed','confirmed_per_100000','deaths','deaths_per_100000']]
covid_data = covid_data.rename(columns={'state':'state_name'})
covid_data.head()

,state_name,county_name,fips_code,lat,lon,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000
0,Alabama,Autauga,01001,32.53952745,-86.64408227,55200.0,1385,2509.06,23,41.67
1,Alabama,Baldwin,01003,30.72774991,-87.72207058,208107.0,4609,2214.73,42,20.18
2,Alabama,Barbour,01005,31.868263,-85.3871286,25782.0,617,2393.14,7,27.15
3,Alabama,Bibb,01007,32.99642064,-87.1251146,22527.0,562,2494.78,6,26.63
4,Alabama,Blount,01009,33.98210918,-86.56790593,57645.0,1071,1857.92,12,20.82


In [6]:
covid_data[['total_population','confirmed','confirmed_per_100000','deaths','deaths_per_100000']].isnull()

,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
3259,True,False,True,False,True
3260,True,False,True,False,True
3261,True,False,True,False,True
3262,True,False,True,False,True


In [7]:
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3264 entries, 0 to 3263
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   state_name            3264 non-null   object
 1   county_name           3264 non-null   object
 2   fips_code             3254 non-null   object
 3   lat                   3200 non-null   object
 4   lon                   3200 non-null   object
 5   total_population      3192 non-null   object
 6   confirmed             3264 non-null   object
 7   confirmed_per_100000  3192 non-null   object
 8   deaths                3264 non-null   object
 9   deaths_per_100000     3192 non-null   object
dtypes: object(10)
memory usage: 255.1+ KB


In [8]:
# Fill NaN values with 0
covid_data = covid_data.fillna(0)

In [9]:
#Check to see if we have all data
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3264 entries, 0 to 3263
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   state_name            3264 non-null   object
 1   county_name           3264 non-null   object
 2   fips_code             3264 non-null   object
 3   lat                   3264 non-null   object
 4   lon                   3264 non-null   object
 5   total_population      3264 non-null   object
 6   confirmed             3264 non-null   object
 7   confirmed_per_100000  3264 non-null   object
 8   deaths                3264 non-null   object
 9   deaths_per_100000     3264 non-null   object
dtypes: object(10)
memory usage: 255.1+ KB


In [10]:
# covid_data['total_population'] = covid_data['total_population'].astype(float)
covid_data = covid_data.astype({'lat': 'float', 'lon': 'float', 'total_population': 'float', 'confirmed': 'int64', 'confirmed_per_100000': 'float', 'deaths': 'int64', 'deaths_per_100000': 'float'})

# have to convert total_population to and int64 (after converting to float)
covid_data = covid_data.astype({'total_population': 'int64'})
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3264 entries, 0 to 3263
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   state_name            3264 non-null   object 
 1   county_name           3264 non-null   object 
 2   fips_code             3264 non-null   object 
 3   lat                   3264 non-null   float64
 4   lon                   3264 non-null   float64
 5   total_population      3264 non-null   int64  
 6   confirmed             3264 non-null   int64  
 7   confirmed_per_100000  3264 non-null   float64
 8   deaths                3264 non-null   int64  
 9   deaths_per_100000     3264 non-null   float64
dtypes: float64(4), int64(3), object(3)
memory usage: 255.1+ KB


In [11]:
# Remove data that has 'fips_cod' of 0 : all values were 0 regardless
covid_data = covid_data.loc[covid_data['fips_code'] != 0]
covid_data

,state_name,county_name,fips_code,lat,lon,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000
0,Alabama,Autauga,01001,32.539527,-86.644082,55200,1385,2509.06,23,41.67
1,Alabama,Baldwin,01003,30.727750,-87.722071,208107,4609,2214.73,42,20.18
2,Alabama,Barbour,01005,31.868263,-85.387129,25782,617,2393.14,7,27.15
3,Alabama,Bibb,01007,32.996421,-87.125115,22527,562,2494.78,6,26.63
4,Alabama,Blount,01009,33.982109,-86.567906,57645,1071,1857.92,12,20.82
...,...,...,...,...,...,...,...,...,...,...
3249,Virginia,Unassigned,90051,0.000000,0.000000,0,0,0.00,0,0.00
3250,Washington,Unassigned,90053,0.000000,0.000000,0,306,0.00,4,0.00
3251,West Virginia,Unassigned,90054,0.000000,0.000000,0,0,0.00,0,0.00
3252,Wisconsin,Unassigned,90055,0.000000,0.000000,0,0,0.00,0,0.00


### Cleaning  the Poverty DF
* Removing the three first rows

In [12]:
# removing the three first rows
poverty_modified = poverty_df.drop(poverty_df.index[0:2])
poverty_modified.head()

,Table with column headers in rows 3 and 4,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87
2,State FIPS code,County FIPS code,Name,State Postal Code,"Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related",...,"Poverty Universe, Age 0-17","Poverty Universe, Age 0-4","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4"
3,00,000,United States,US,"319,184,033","52,529,919","72,163,269","19,301,529","317,741,588","52,669,201",...,"71,741,141","19,181,906",276207757,51642359,71684956,18968750,271059449,51060953,71338364,19382484
4,01,000,Alabama,AL,"4,763,811","781,913","1,069,994","284,188","4,752,519","790,771",...,"1,104,080","296,196",4368014,804291,1120718,293558,4348444,789510,1088427,295264
5,01,001,Autauga County,AL,"55,073","9,677","12,987",NaN,"55,021","9,911",...,"12,377",NaN,43711,9245,12507,NaN,43524,8856,12148,NaN
6,01,003,Baldwin County,AL,"215,255","34,508","46,265",NaN,"209,922","34,058",...,"34,503",NaN,139273,25048,34302,NaN,136585,24609,33859,NaN


In [13]:
new_header = poverty_modified.iloc[0]
poverty_nh = poverty_modified[1:]
poverty_nh.columns =new_header
poverty_nh.head()

2,State FIPS code,County FIPS code,Name,State Postal Code,"Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related",...,"Poverty Universe, Age 0-17","Poverty Universe, Age 0-4","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4","Poverty Universe, All Ages","Poverty Universe, Age 5-17 related","Poverty Universe, Age 0-17","Poverty Universe, Age 0-4"
3,00,000,United States,US,"319,184,033","52,529,919","72,163,269","19,301,529","317,741,588","52,669,201",...,"71,741,141","19,181,906",276207757,51642359,71684956,18968750,271059449,51060953,71338364,19382484
4,01,000,Alabama,AL,"4,763,811","781,913","1,069,994","284,188","4,752,519","790,771",...,"1,104,080","296,196",4368014,804291,1120718,293558,4348444,789510,1088427,295264
5,01,001,Autauga County,AL,"55,073","9,677","12,987",NaN,"55,021","9,911",...,"12,377",NaN,43711,9245,12507,NaN,43524,8856,12148,NaN
6,01,003,Baldwin County,AL,"215,255","34,508","46,265",NaN,"209,922","34,058",...,"34,503",NaN,139273,25048,34302,NaN,136585,24609,33859,NaN
7,01,005,Barbour County,AL,"21,979","3,848","5,106",NaN,"22,224","3,901",...,"7,148",NaN,26480,5422,7341,NaN,25482,5138,6966,NaN


In [14]:
# selecting the columns of interest by index: 'State FIPS code', 'County FIPS code', 'Name', 'State Postal Code', 'Poverty Universe, All Ages'
poverty_select = poverty_nh.iloc[:, 0:5]
poverty_select.head()

2,State FIPS code,County FIPS code,Name,State Postal Code,"Poverty Universe, All Ages"
3,00,000,United States,US,"319,184,033"
4,01,000,Alabama,AL,"4,763,811"
5,01,001,Autauga County,AL,"55,073"
6,01,003,Baldwin County,AL,"215,255"
7,01,005,Barbour County,AL,"21,979"


In [15]:
# removing state names..... later
poverty_drop = poverty_select.loc[poverty_select['County FIPS code'] != '000']
poverty_drop.head()

2,State FIPS code,County FIPS code,Name,State Postal Code,"Poverty Universe, All Ages"
5,01,001,Autauga County,AL,"55,073"
6,01,003,Baldwin County,AL,"215,255"
7,01,005,Barbour County,AL,"21,979"
8,01,007,Bibb County,AL,"20,212"
9,01,009,Blount County,AL,"57,238"


In [16]:
# combining the two FIPS columns to get the StateCounty full FIPS code:
poverty_drop['fips_code'] = poverty_drop['State FIPS code'] + poverty_drop['County FIPS code']
poverty_drop.head()

/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


2,State FIPS code,County FIPS code,Name,State Postal Code,"Poverty Universe, All Ages",fips_code
5,01,001,Autauga County,AL,"55,073",01001
6,01,003,Baldwin County,AL,"215,255",01003
7,01,005,Barbour County,AL,"21,979",01005
8,01,007,Bibb County,AL,"20,212",01007
9,01,009,Blount County,AL,"57,238",01009


In [17]:
# selecting the columns the final data frame
poverty_trim = poverty_drop.iloc[:, 2:6]

In [18]:
poverty_trim = poverty_trim.rename(columns={'Poverty Universe, All Ages':'poverty',
                                            'State Postal Code': 'state_code',
                                            'Name': 'county_name'})
poverty_trim.head()

2,county_name,state_code,poverty,fips_code
5,Autauga County,AL,"55,073",01001
6,Baldwin County,AL,"215,255",01003
7,Barbour County,AL,"21,979",01005
8,Bibb County,AL,"20,212",01007
9,Blount County,AL,"57,238",01009


In [19]:
poverty_trim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3149 entries, 5 to 3203
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   county_name  3149 non-null   object
 1   state_code   3149 non-null   object
 2   poverty      3141 non-null   object
 3   fips_code    3149 non-null   object
dtypes: object(4)
memory usage: 123.0+ KB


In [20]:
# exploring null values
null_val = poverty_trim[poverty_trim['poverty'].isnull()]
null_val

2,county_name,state_code,poverty,fips_code
95,Prince of Wales-Outer Ketchikan Census Area,AK,NaN,02201
98,Skagway-Hoonah-Angoon Census Area,AK,NaN,02232
101,Wade Hampton Census Area,AK,NaN,02270
103,Wrangell-Petersburg Census Area,AK,NaN,02280
568,Kalawao County,HI,NaN,15005
2468,Shannon County,SD,NaN,46113
2972,Bedford city,VA,NaN,51515
2977,Clifton Forge,VA,NaN,51560


In [21]:
poverty_trim['poverty'].isnull().sum()

8

In [22]:
# replacing NaN values with 0
poverty_trim['poverty'] = poverty_trim['poverty'].fillna(0)
poverty_trim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3149 entries, 5 to 3203
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   county_name  3149 non-null   object
 1   state_code   3149 non-null   object
 2   poverty      3149 non-null   object
 3   fips_code    3149 non-null   object
dtypes: object(4)
memory usage: 123.0+ KB


In [23]:
# removing leading/reailing speaces on poverty column
poverty_trim['poverty'] = poverty_trim['poverty'].str.strip()

In [24]:
# removing commas as thousand separator
poverty_trim['poverty'] = poverty_trim['poverty'].replace(',','', regex=True)
poverty_trim.head()

2,county_name,state_code,poverty,fips_code
5,Autauga County,AL,55073,01001
6,Baldwin County,AL,215255,01003
7,Barbour County,AL,21979,01005
8,Bibb County,AL,20212,01007
9,Blount County,AL,57238,01009


In [25]:
poverty_trim.dtypes

2
county_name    object
state_code     object
poverty        object
fips_code      object
dtype: object

In [26]:
# besides replacing the NaN with 0, the dataset was still showing nul-values.
poverty_data = poverty_trim.dropna()
poverty_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3141 entries, 5 to 3203
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   county_name  3141 non-null   object
 1   state_code   3141 non-null   object
 2   poverty      3141 non-null   object
 3   fips_code    3141 non-null   object
dtypes: object(4)
memory usage: 122.7+ KB


In [27]:
poverty_data = poverty_data.astype({'poverty': 'int64'})
poverty_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3141 entries, 5 to 3203
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   county_name  3141 non-null   object
 1   state_code   3141 non-null   object
 2   poverty      3141 non-null   int64 
 3   fips_code    3141 non-null   object
dtypes: int64(1), object(3)
memory usage: 122.7+ KB


In [28]:
poverty_data.head()

2,county_name,state_code,poverty,fips_code
5,Autauga County,AL,55073,01001
6,Baldwin County,AL,215255,01003
7,Barbour County,AL,21979,01005
8,Bibb County,AL,20212,01007
9,Blount County,AL,57238,01009


## Connect to Postgres SQL to Load Data

In [29]:
rds_connection_string = "postgres:" + post_gres_pw + "@localhost:5432/covid_poverty_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [30]:
#check to see if we have tables
engine.table_names()

['covid_data', 'poverty_data']

In [31]:
# Load COVID data to SQL
covid_data.to_sql(name='covid_data', con=engine, if_exists='append', index=False)

In [32]:
# Test to see if covid data was entered
pd.read_sql_query('select * from covid_data', con=engine).head()

,fips_code,state_name,county_name,lat,lon,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000
0,01001,Alabama,Autauga,33,-87,55200,1385,2509,23,42
1,01003,Alabama,Baldwin,31,-88,208107,4609,2215,42,20
2,01005,Alabama,Barbour,32,-85,25782,617,2393,7,27
3,01007,Alabama,Bibb,33,-87,22527,562,2495,6,27
4,01009,Alabama,Blount,34,-87,57645,1071,1858,12,21


In [33]:
# Load poverty data to SQL
poverty_data.to_sql(name='poverty_data', con=engine, if_exists='append', index=False)

In [34]:
# Test to see if poverty data was entered
pd.read_sql_query('select * from poverty_data', con=engine).head()

,fips_code,state_code,county_name,poverty
0,01001,AL,Autauga County,55073
1,01003,AL,Baldwin County,215255
2,01005,AL,Barbour County,21979
3,01007,AL,Bibb County,20212
4,01009,AL,Blount County,57238
